# Problem 3

In [1]:
import random
import numpy as np
from skimage.io import imread
from skimage.color import gray2rgb
from skimage.measure import block_reduce
from scipy.misc import imresize
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


## Load the image data

In [2]:
def load(images, labels, image_name, image_class):
    
    working_path = "./image/" + image_name
    jpeg_files_path = glob.glob(os.path.join(working_path, '*.JPEG'))
    
    for path in jpeg_files_path:
        origin_image = imread(path)
        if len(origin_image.shape) == 3:
            resize_image = imresize(origin_image, (64,64,3))
            images.append(resize_image)
            labels.append(image_class)

In [3]:
# Get the file path for all 8987 images
import os, glob
images = []
labels = []

# plant - 0
load(images, labels, "n00017222", 0)

# geological - 1
load(images, labels, "n09287968", 1)

# fungus - 2
load(images, labels, "n12992868", 2)

# sport - 3
load(images, labels, "n00523513", 3)

# person - 4
load(images, labels, "n00007846", 4)

# animal - 5
load(images, labels, "n00015388", 5)

### Split data

In [4]:
images = np.array(images)
labels = np.array(labels)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 6240)

In [5]:
x_train_norm = x_train.astype('float32') / 255.
x_test_norm = x_test.astype('float32') / 255.

## Convert the images to vectors

### 1. Autoencoder

In [6]:
input_img = Input(shape=(64, 64, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [7]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train_norm, x_train_norm,
                epochs=20,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_norm, x_test_norm),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 7178 samples, validate on 1795 samples
Epoch 1/20
7178/7178 [==============================] - 108s - loss: 0.6638 - val_loss: 0.6107

In [8]:
# Get the reconstructed train set from autoencoder
x_train_auto = encoder.predict(x_train_norm)

# Get the reconstructed test set from autoencoder
x_test_auto = encoder.predict(x_test_norm)

### 2. SVD

In [9]:
# Reshape train and test data
x_train_flat = x_train.reshape(x_train.shape[0], np.prod(x_train.shape[1:]))
x_test_flat = x_test.reshape(x_test.shape[0], np.prod(x_test.shape[1:]))

In [39]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=30)
x_train_svd = svd.fit_transform(x_train_flat)
x_test_svd = svd.transform(x_test_flat)

We test different values of $k$. According to the values of the diagonal element in Sigma, we decide to choose $k$ = 30.

### 3. RGB

In [33]:
def toVector(image):
    result = []
    for layer in range(3):
        arr, bins = np.histogram(image[:, :, layer], 256, [0, 255])
        result = result + arr.tolist()
    return result

In [34]:
x_train_rgb = []
for elm in x_train:
    x_train_rgb.append(toVector(elm))
x_train_rgb = np.array(x_train_rgb)

x_test_rgb = []
for elm in x_test:
    x_test_rgb.append(toVector(elm))
x_test_rgb = np.array(x_test_rgb)

### 4. HSV

In [36]:
from skimage import color

x_train_hsv = []
for elm in x_train:
    x_train_hsv.append(toVector(color.rgb2hsv(elm) * 255))
x_train_hsv = np.array(x_train_hsv)

x_test_hsv = []
for elm in x_test:
    x_test_hsv.append(toVector(color.rgb2hsv(elm) * 255))
x_test_hsv = np.array(x_test_hsv)

## Compare classification results

In [48]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import pearsonr
def pearson_distance(test, train):
    row = test.shape[0]
    col = train.shape[0]
    result = np.zeros((row, col))
    for i in range(row):
        for j in range(col):
            result[i, j] = pearsonr(test[i, :], train[j, :])[0]
    return result

In [61]:
def topK(matrix, method, k): # for pearson, method = -1， for euclidean, k = 1
    top_k = np.zeros((matrix.shape[0], k))
    
    for i in range(matrix.shape[0]):
        arr = matrix[i, :]
        index = np.argsort(arr * method)[:k]
        top_k[i, :] = index
    
    top_k = top_k.astype('int')
    return top_k

In [62]:
def majority(top_k, label):
    count = [0] * 6
    for elm in label[top_k]:
        count[elm] += 1
    return count.index(max(count))

In [67]:
def computeAcc(y_test, y_train, matrix, method, k): # for pearson, method = -1， for euclidean, k = 1
    top_k = topK(matrix, method, k)
    
    prediction = []
    total = 0
    
    n = y_test.shape[0]
    for i in range(n):
        prediction.append(majority(top_k[i], y_train))
        if prediction[i] == y_test[i]:
            total += 1
    prediction = np.array(prediction) 
    
    return prediction, (total * 100.0 / n)

In [55]:
from sklearn.metrics import confusion_matrix

### 1. Autoencoder

In [42]:
x_train_auto = x_train_auto.reshape(x_train_auto.shape[0], np.prod(x_train_auto.shape[1:]))
x_test_auto = x_test_auto.reshape(x_test_auto.shape[0], np.prod(x_test_auto.shape[1:]))

In [45]:
# Euclidean distance
euclidean_matrix_auto = euclidean_distances(x_test_auto, x_train_auto)

In [49]:
# Pearson distance
pearson_matrix_auto = pearson_distance(x_test_auto, x_train_auto)

In [68]:
pre_euc_auto, acc_euc_auto = computeAcc(y_test, y_train, euclidean_matrix_auto, 1, 5)
pre_pea_auto, acc_pea_auto = computeAcc(y_test, y_train, pearson_matrix_auto, -1, 5)

confusion_euc_auto = confusion_matrix(y_test, pre_euc_auto)
confusion_pea_auto = confusion_matrix(y_test, pre_pea_auto)

In [70]:
print("--------------- Result for Euclidean distance ---------------")
print(confusion_euc_auto)
print("Accuracy is " + str(acc_euc_auto) + "%")

print("--------- Result for Pearson correlation coefficient ---------")
print(confusion_pea_auto)
print("Accuracy is " + str(acc_pea_auto) + "%")

--------------- Result for Euclidean distance ---------------
[[132  64  32   7   9  24]
 [ 35 297   5   8   5  16]
 [ 54  67  70   7   4  25]
 [ 53 177  15  70   7  40]
 [ 18 122  19   9  40  30]
 [ 60 149  22  19  18  66]]
Accuracy is 37.60445682451253%
--------- Result for Pearson correlation coefficient ---------
[[135  67  25  10  10  21]
 [ 22 297   8  11   9  19]
 [ 52  62  79  10   7  17]
 [ 41 171  17  88  11  34]
 [ 17 103  18  11  58  31]
 [ 49 137  29  18  29  72]]
Accuracy is 40.61281337047354%


### 2. SVD

In [71]:
# Euclidean distance
euclidean_matrix_svd = euclidean_distances(x_test_svd, x_train_svd)

In [72]:
# Pearson distance
pearson_matrix_svd = pearson_distance(x_test_svd, x_train_svd)

In [73]:
pre_euc_svd, acc_euc_svd = computeAcc(y_test, y_train, euclidean_matrix_svd, 1, 5)
pre_pea_svd, acc_pea_svd = computeAcc(y_test, y_train, pearson_matrix_svd, -1, 5)

confusion_euc_svd = confusion_matrix(y_test, pre_euc_svd)
confusion_pea_svd = confusion_matrix(y_test, pre_pea_svd)

In [74]:
print("--------------- Result for Euclidean distance ---------------")
print(confusion_euc_svd)
print("Accuracy is " + str(acc_euc_svd) + "%")

print("--------- Result for Pearson correlation coefficient ---------")
print(confusion_pea_svd)
print("Accuracy is " + str(acc_pea_svd) + "%")

--------------- Result for Euclidean distance ---------------
[[134  38  43  23   9  21]
 [ 36 237  23  48  10  12]
 [ 73  26  86  20   3  19]
 [ 45 107  27 148   9  26]
 [ 28  79  38  36  40  17]
 [ 83  89  45  63  14  40]]
Accuracy is 38.16155988857939%
--------- Result for Pearson correlation coefficient ---------
[[131  36  42  25   6  28]
 [ 18 262  16  47   7  16]
 [ 74  49  63  18   8  15]
 [ 38 110  20 157   4  33]
 [ 21  83  21  33  46  34]
 [ 68 105  34  60  15  52]]
Accuracy is 39.610027855153206%


### 3. RGB

In [75]:
# Euclidean distance
euclidean_matrix_rgb = euclidean_distances(x_test_rgb, x_train_rgb)

In [76]:
# Pearson distance
pearson_matrix_rgb = pearson_distance(x_test_rgb, x_train_rgb)

In [77]:
pre_euc_rgb, acc_euc_rgb = computeAcc(y_test, y_train, euclidean_matrix_rgb, 1, 5)
pre_pea_rgb, acc_pea_rgb = computeAcc(y_test, y_train, pearson_matrix_rgb, -1, 5)

confusion_euc_rgb = confusion_matrix(y_test, pre_euc_rgb)
confusion_pea_rgb = confusion_matrix(y_test, pre_pea_rgb)

In [78]:
print("--------------- Result for Euclidean distance ---------------")
print(confusion_euc_rgb)
print("Accuracy is " + str(acc_euc_rgb) + "%")

print("--------- Result for Pearson correlation coefficient ---------")
print(confusion_pea_rgb)
print("Accuracy is " + str(acc_pea_rgb) + "%")

--------------- Result for Euclidean distance ---------------
[[139  26  64   9  13  17]
 [ 49 175  60  27  22  33]
 [ 32  16 151   8   7  13]
 [ 45  69  79 109  17  43]
 [ 27  54  61  26  44  26]
 [ 82  53 104  28  20  47]]
Accuracy is 37.04735376044568%
--------- Result for Pearson correlation coefficient ---------
[[149  30  50   9  11  19]
 [ 50 218  42  11  17  28]
 [ 51  35 105   5  11  20]
 [ 50  87  68  99  19  39]
 [ 34  61  46  19  63  15]
 [ 82  83  70  27  25  47]]
Accuracy is 37.938718662952645%


### 4. HSV

In [79]:
# Euclidean distance
euclidean_matrix_hsv = euclidean_distances(x_test_hsv, x_train_hsv)

In [80]:
# Pearson distance
pearson_matrix_hsv = pearson_distance(x_test_hsv, x_train_hsv)

In [81]:
pre_euc_hsv, acc_euc_hsv = computeAcc(y_test, y_train, euclidean_matrix_hsv, 1, 5)
pre_pea_hsv, acc_pea_hsv = computeAcc(y_test, y_train, pearson_matrix_hsv, -1, 5)

confusion_euc_hsv = confusion_matrix(y_test, pre_euc_hsv)
confusion_pea_hsv = confusion_matrix(y_test, pre_pea_hsv)

In [82]:
print("--------------- Result for Euclidean distance ---------------")
print(confusion_euc_hsv)
print("Accuracy is " + str(acc_euc_hsv) + "%")

print("--------- Result for Pearson correlation coefficient ---------")
print(confusion_pea_hsv)
print("Accuracy is " + str(acc_pea_hsv) + "%")

--------------- Result for Euclidean distance ---------------
[[178  23  42   6   2  17]
 [ 26 185  60  37  10  48]
 [ 34  22 139  17   0  15]
 [ 48  46  72 164   6  26]
 [ 19  43  50  46  64  16]
 [ 78  60 101  38   9  48]]
Accuracy is 43.34261838440111%
--------- Result for Pearson correlation coefficient ---------
[[177  31  33   8   6  13]
 [ 20 227  42  22  15  40]
 [ 31  39 127  11   5  14]
 [ 52  72  61 117  19  41]
 [ 14  67  43  29  70  15]
 [ 76  86  86  20  17  49]]
Accuracy is 42.729805013927574%
